# 筆記本 2：使用大語言模型進行對話處理

在這個教學範例中，我們會學習如何根據不同的 API 供應商（Google 或 OpenAI）使用大語言模型來處理對話記錄。

### 目標
- 讀取歷史對話記錄。
- 使用大語言模型進行互動，如 Google 的 Gemini 或 OpenAI 的 GPT-4。
- 儲存模型生成的回應。

此範例將幫助你透過 API 的方式與大語言模型進行互動，並自訂系統設定檔讓模型產生滿足你的需求的答案。

### 必要套件
首先，我們需要安裝兩個必要的 API 套件：
- `google.generativeai`: 用來與 Google 的大語言模型互動（例如 Gemini）。
- `openai`: 用來與 OpenAI 的 GPT-4 等模型進行互動。

使用以下命令來安裝這些套件：
```bash
!pip install google-generativeai openai
```

In [ ]:
# 安裝相關套件
!pip install google-generativeai openai

### 步驟 1: 定義命令行參數解析模組
我們首先定義一個自訂的解析器來處理命令行參數，這將幫助我們從命令行讀取相關的選項，如 API 金鑰、模型名稱、系統設定檔等等。這些參數將會被用來在後續的步驟中做處理。


In [ ]:
import argparse
import os
import json

class CustomArgumentParser(argparse.ArgumentParser):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def print_help(self, file=None):
        # 自訂顯示的幫助訊息
        help_message = f"""使用方法: {self.prog} [-h] -K 金鑰 -L 大語言模型 -S 系統設定檔 -H 對話記錄檔案 -I 輸入檔案 -O 輸出檔案 -P API供應商{{google或openai}}

使用大語言模型處理對話記錄

選項:
  -h, --help            顯示此幫助訊息並退出
  -K KEY, --key KEY     API 金鑰
  -L LLM, --llm LLM     大語言模型的名稱 (例如 'gemini-1.5-pro' 或 'gpt-4o')
  -S SYSTEM, --system SYSTEM
                        系統設定檔 (JSON 格式)
  -H HISTORY, --history HISTORY
                        對話記錄檔案 (JSON 格式)
  -I INPUT, --input INPUT
                        輸入檔案
  -O OUTPUT, --output OUTPUT
                        輸出檔案
  -P {{google,openai}}, --provider {{google,openai}}
                        選擇使用的 API 供應商 ('google' 或 'openai')
        """
        print(help_message, file=file)

### 步驟 2: 定義主程序
這個主程序的功能是：從命令參數中讀取設定，根據指定的 API 供應商（Google 或 OpenAI）來進行兩種不同的處理流程，應用對應的大語言模型。

#### 步驟包括
1. **讀取系統設定檔**：這是設定如何調耳大語言模型的參數，比如溫度（temperature）等生成設置。
2. **讀取對話記錄**：歷史聊天記錄是一個可選的參數，我們可以使用它來提供上下文給模型。
3. **使用模型生成回應**：根據指定的 API 供應商來發送請求，並進行模型的回應生成。
4. **保存結果**：將生成的回應寫入指定的輸出檔案中。

In [ ]:
import sys
import google.generativeai as genai
import openai

def main():
    # 使用 CustomArgumentParser 處理命令列參數
    parser = CustomArgumentParser(description="使用大語言模型處理對話記錄")

    # 添加命令列選項
    parser.add_argument("-K", "--key", required=True, help="API 金鑰")
    parser.add_argument("-L", "--llm", required=True,
                        help="處理的模型名稱 (例如 'gemini-1.5-pro' 或 'gpt-4o')")
    parser.add_argument("-S", "--system", required=True,
                        help="系統設定檔 (JSON 格式)")
    parser.add_argument("-H", "--history", required=True,
                        help="對話記錄檔案 (JSON 格式)")
    parser.add_argument("-I", "--input", required=True, help="輸入檔案")
    parser.add_argument("-O", "--output", required=True, help="輸出的檔案")
    parser.add_argument("-P", "--provider", required=True,
                        choices=['google', 'openai'], help="選擇使用的 API 供應商 ('google' 或 'openai')")

    args = parser.parse_args()

    # 讀取系統設定檔 (JSON 格式)
    with open(args.system, "r", encoding="utf-8") as system_file:
        system_config = json.load(system_file)
        system_instruction = system_config.get("instruction", "")

    # 讀取對話歷史 (JSON 格式)
    with open(args.history, "r", encoding="utf-8") as history_file:
        history = json.load(history_file)

    # 讀取輸入檔案 (純文字格式)
    with open(args.input, "r", encoding="utf-8") as input_file:
        message = input_file.read()

    # 根據不同的 API 供應商進行對話
    if args.provider == 'google':
        # 使用 Google API 的處理方式
        genai.configure(api_key=args.key)

        # 設定模型配置
        generation_config = {
            "temperature": system_config.get("temperature", 1),
            "top_p": system_config.get("top_p", 0.95),
            "top_k": system_config.get("top_k", 64),
            "max_output_tokens": system_config.get("max_output_tokens", 8192),
            "response_mime_type": system_config.get("response_mime_type", "text/plain")
        }

        model = genai.GenerativeModel(
            model_name=args.llm,
            generation_config=generation_config,
            system_instruction=system_instruction,
        )

        # 開始對話
        chat_session = model.start_chat(history=history)

        # 送出訊息並取得回應
        response = chat_session.send_message({
            "role": "user",
            "parts": [message]
        })

        generated_message = response.text
    
    elif args.provider == 'openai':
        # 使用 OpenAI API 的處理方式
        client = openai.OpenAI(api_key=args.key)

        msg_list = [{"role": "system", "content": system_instruction}
                    ] + history + [{"role": "user", "content": message}]
        
        # 設定模型配置
        generation_config = {
            "temperature": system_config.get("temperature", 1),
            "top_p": system_config.get("top_p", 0.95),
            "top_k": system_config.get("top_k", 64),
            "max_output_tokens": system_config.get("max_output_tokens", 8192),
            "response_mime_type": system_config.get("response_mime_type", "text/plain"),
            "frequency_penalty": system_config.get("frequency_penalty", 0),
            "presence_penalty": system_config.get("presence_penalty", 0),
            "response_format": system_config.get("response_format", {"type": "text"})
        }        

        # 開始對話
        response = client.chat.completions.create(
            model=args.llm,
            messages=msg_list,
            temperature=generation_config["temperature"],
            max_tokens=generation_config["max_output_tokens"],
            top_p=generation_config["top_p"],
            frequency_penalty=generation_config["frequency_penalty"],
            presence_penalty=generation_config["presence_penalty"],
            response_format=generation_config["response_format"]
        )

        generated_message = response.choices[0].message.content

    # 將回應保存至輸出檔案
    with open(args.output, "w", encoding="utf-8") as output_file:
        output_file.write(generated_message)

    print(f"\n\r回應已儲存至 {args.output}")

if __name__ == "__main__":

    # 模擬命令列參數
    # for google
    # sys.argv = ['02_extract_event_mentions.py', '-K', 'YOUR_KEY', '-L', 'gemini-1.5-pro', '-S', 'system_config_google.json', '-H', 'history_google.json', '-I', 'output_summary.txt', '-O', 'event_log.txt', '-P', 'google']
    # for openai
    sys.argv = ['02_extract_event_mentions.py', '-K', 'YOUR_KEY', '-L', 'gpt-4o', '-S', 'system_config_openai.json', '-H', 'history_openai.json', '-I', 'output_summary.txt', '-O', 'event_log.txt', '-P', 'openai']

    main()

### 結論
- 在這個範例中，我們學習了如何根據不同的語言模型提供商（Google 和 OpenAI）來和大語言模型進行互動。
- 我們的程式是高度可配置的，可以根據自己的需求調整語言模型、溫度、回應生成的最大字數、對話歷史等等。
- 接下來，你可以使用這些工具來進行更深入的自然語言處理應用。